In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from argparse import Namespace
import misc.logging_utils as logging_utils

args = Namespace()
logger = logging_utils.get_ipython_logger()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns; sns.set(style='white')

import misc.bio_utils.bed_utils as bed_utils
import misc.mpl_utils as mpl_utils
import riboutils.ribo_utils as ribo_utils

In [ ]:
args.orfs = "/path/to/my/filtered.predicted-orfs.bed.gz"
args.out = "/path/to/my/filtered.predicted-orfs.bar-chart.pdf"
args.use_groups = False
args.title = "My title"

args.fontsize = 20
args.legend_fontsize = 15

In [ ]:
msg = "Reading bed file"
logger.info(msg)

bed = bed_utils.read_bed(args.orfs)

if args.use_groups:
    bed['orf_type_group'] = bed['orf_type'].map(ribo_utils.orf_type_labels_reverse_mapping)
    orf_type_counts = bed.groupby(['orf_type_group', 'strand']).size()
    orf_type_counts = orf_type_counts.reset_index(name="count")
    orf_type_counts['display_name'] = orf_type_counts['orf_type_group'].map(ribo_utils.orf_type_labels_display_name_map)
else:
    orf_type_counts = bed.groupby(['orf_type', 'strand']).size()
    orf_type_counts = orf_type_counts.reset_index(name="count")
    orf_type_counts['display_name'] = orf_type_counts['orf_type'].map(ribo_utils.orf_type_display_name_map)

msg = "Creating the bar chart"

color = sns.palettes.color_palette("Set3", n_colors=3)

fig, ax = plt.subplots(figsize=(9,5))
sns.barplot(
    x="display_name",
    y="count",
    hue="strand",
    data=orf_type_counts,
    ax=ax,
    zorder=-1,
    palette='Set3'
)

sns.despine()

ax.legend(
    loc='upper right', 
    bbox_to_anchor=(1.0, 1.1), 
    fontsize=args.legend_fontsize, 
    frameon=True, 
    framealpha=0.9,
    title="Strand"
)
mpl_utils.set_legend_title_fontsize(ax, args.fontsize)
#ax.legend_.remove()

ax.set_yscale('log')
ax.set_ylim((1, 1e4))

ax.set_ylabel("Number of ORFs", fontsize=args.fontsize)
ax.set_xlabel("", fontsize=0)

# rotate the ORF type names
mpl_utils.set_ticklabels_fontsize(ax, args.fontsize)
mpl_utils.set_ticklabel_rotation(ax, axis='x', rotation=90)

# place the ORF type names in the middle of the bar
for ticklabel in ax.xaxis.get_ticklabels():    
    p = ticklabel.get_position()
    ticklabel.set_position((p[0], 0.1))
    ticklabel.set_verticalalignment('bottom')
    
if args.title is not None:
    ax.set_title(args.title, fontsize=args.fontsize)
    
if args.out is not None:
    fig.savefig(args.out, bbox_inches='tight')